In [112]:
## récupérer les données
import xml.etree.ElementTree as ET
tree = ET.parse("DATA/Corpus_rapv2.xml")
root = tree.getroot()
liste_artistes = []
liste_paroles = []
liste_titres = []
for child in root:##
    artiste = child.get("artiste")
    liste_artistes.append(artiste)
    liste_titres.append(child.get("titre"))
    
for chanson in root.findall("chanson"):
    paroles = chanson.find("paroles")
    liste_paroles.append(paroles.text)
    
print(len(liste_paroles))

200


In [111]:
##focntions de pré-traitement et d'évaluation
import nltk
from nltk.corpus import stopwords 
import re

def remove_stopwords(chaine):
  final_stopwords_list =stopwords.words('french')
  s = chaine
  for stopword in final_stopwords_list:
    chaine = re.sub(f" {stopword} ", " ", chaine)
  return chaine

def remove_ponctuation(chaine):
  ponctuations = [",", "'", '"', "-", "\."]
  for stopword in ponctuations:
    chaine = re.sub(f" {stopword} ", " ", chaine)
  return chaine

def resultats_par_classe(y_test, y_pred):
  dic_resultats = {} #pour recenser les erreurs et les bonnnes prédictions
  for artiste in set(y_test):
    dic_resultats[artiste] = {"VP":0, "FP":0, "FN":0}
    
  for i in range(len(y_test)):
    if y_test[i] == y_pred[i]:#vrai positif
        dic_resultats[y_test[i]]["VP"]+=1#on ajoute 1, équivalent à :
    else:
        dic_resultats[y_test[i]]["FN"]+=1 # Il en manque un ici (FN)
        dic_resultats[y_pred[i]]["FP"]+=1 # Il y en a un de trop là (FP)
  return dic_resultats  


In [120]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Perceptron

liste_chansons = []
for i in range(len(liste_titres)):
    chanson = liste_titres[i] +"\n"+liste_paroles[i]
    liste_chansons.append(chanson)


In [116]:

y = liste_artistes
### Vectoriser
from sklearn.feature_extraction.text import CountVectorizer
V = CountVectorizer(ngram_range = (1,3))

for pretraitement in ["stopwords", "e", "ponctuation", "None"]:
  if pretraitement =="stopwords":
    liste_titres_pretraite = [remove_stopwords(titre) for titre in liste_paroles]
    X = V.fit_transform(liste_titres_pretraite)
  elif pretraitement =="e":
    liste_titres_pretraite = [re.sub("e", "", titre) for titre in liste_paroles]
    X = V.fit_transform(liste_titres_pretraite)
  elif pretraitement =="ponctuation":
    liste_titres_pretraite = [remove_ponctuation(titre) for titre in liste_paroles]
    X = V.fit_transform(liste_titres_pretraite)    
  else:
    X = V.fit_transform(liste_paroles)
  for split_size in [0.3]: #[0.1, 0.2, 0.3, 0.9]:
    print(f"Split_size : {split_size}, Pretraitement: {pretraitement}")
    #découpage train VS test
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = split_size, random_state=0)
    ##classification
    ppn = Perceptron(eta0 = 0.1, random_state=0)
    ppn.fit(X_train, y_train)
    y_pred = ppn.predict(X_test)
    print ('Accuracy: %.2f' % accuracy_score(y_test, y_pred))
    res_par_classe = resultats_par_classe(y_test, y_pred)
    for classe, res in res_par_classe.items():
        print(f" {classe} : {res}")


92i Veyron

[Intro]
Personne, personne
Personne, personne

[Couplet 1]
Nouveau riche, ma Lamborghini a pris quelques dos-d'âne
J'fais ni la queue au Ritz, ni au McDonald's
Si c'est eux qui ont raison, je n'suis pas raisonnable
La rafale dans ton salon sera sûrement désagréable
Nouveau riche, ma Lamborghini a pris quelques dos-d'âne
J'fais ni la queue au Ritz, ni au McDonald's
Si c'est eux qui ont raison, je n'suis pas raisonnable
La rafale dans ton salon sera sûrement désagréable

[Refrain]
On trinque à nos balafres, à nos crochets tous les soirs
"Noir c'est noir", ont-ils dit, y'a donc vraiment plus d'espoir
Les vainqueurs l'écrivent, les vaincus racontent l'histoire
Les vainqueurs l'écrivent, les vaincus racontent l'histoire
Personne dans le monde ne marche du même pas
Leurs règles ont toutes une tombe, c'est ça qu'ils n'comprennent pas
Des allers-retours en prison, certains n'en reviennent pas
J'ai une kalash et des mouftons, tous sous le même toit
Toujours d'humeur à les lever, que

ZeroDivisionError: division by zero